In [1]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [20]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

### Instantiate our model object

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
llm.invoke("Hello, how are you?")

AIMessage(content="I'm functioning properly, thank you for asking. I'm a large language model, so I don't have emotions or feelings like humans do, but I'm here to help answer any questions or provide information you might need. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 41, 'total_tokens': 94, 'completion_time': 0.070666667, 'prompt_time': 0.007701828, 'queue_time': 0.06578560600000001, 'total_time': 0.078368495}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a56f6eea01', 'finish_reason': 'stop', 'logprobs': None}, id='run-b140fae6-e49d-4c24-9425-4a46c3b0e597-0', usage_metadata={'input_tokens': 41, 'output_tokens': 53, 'total_tokens': 94})

In [7]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='The translation of "I love programming" to French is:\n\n"J\'adore le programmation."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 55, 'total_tokens': 77, 'completion_time': 0.029333333, 'prompt_time': 0.011183663, 'queue_time': 2.9522059869999997, 'total_time': 0.040516996}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a56f6eea01', 'finish_reason': 'stop', 'logprobs': None}, id='run-f45ceadd-900b-4733-b167-f81879118f1b-0', usage_metadata={'input_tokens': 55, 'output_tokens': 22, 'total_tokens': 77})

In [9]:
from langchain_core.prompts import ChatPromptTemplate

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are english to german language translator. Translate the user sentence."),
    ("user", "{input}")],
)

In [22]:
from pydantic import BaseModel, Field

In [25]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

@tool
def divide(a: int, b: int) -> float:
    """Divide a by b."""
    return a / b

@tool
def add(a: int, b: int) -> int:
    """Add a and b."""
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract b from a."""    
    return a - b

@tool
def floor_divide(a: int, b: int) -> int:
    """Divide a by b and round down."""
    return a // b

In [28]:
tools = [multiply, divide, add, subtract, floor_divide]

model_with_tools = llm.bind_tools(tools)

model_with_tools.invoke("add 2 and 3")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h9g3', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 448, 'total_tokens': 466, 'completion_time': 0.024, 'prompt_time': 0.014816846, 'queue_time': 0.048062519000000005, 'total_time': 0.038816846}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7e46e430-7388-4fbc-949d-f36cc8f0e1d3-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 3}, 'id': 'call_h9g3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 448, 'output_tokens': 18, 'total_tokens': 466})

In [29]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why did the cat join a band?', punchline='Because it wanted to be the purr-cussionist!', rating=8)

In [33]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class TemperatureSensor(BaseModel):
    sensor_id: str = Field(description="Unique identifier for the sensor")
    location: str = Field(description="Location of the sensor")
    temperature: float = Field(description="Current temperature reading from the sensor")
    unit: str = Field(default="Celsius", description="Unit of the temperature reading")
    timestamp: datetime = Field(default_factory=datetime.now, description="Timestamp of the reading")
    status: Optional[str] = Field(default="active", description="Status of the sensor (e.g., active, inactive, maintenance)")

structured_llm = llm.with_structured_output(TemperatureSensor)

structured_llm.invoke("Based on weather condition of specific date and day, what would be the temperature of mumbai in Celsius?")

TemperatureSensor(sensor_id='Mumbai_Temp', location='Mumbai', temperature=28.0, unit='Celsius', timestamp=datetime.datetime(2023, 3, 17, 0, 0), status='active')

### Bug Logger POC

In [36]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class TechnicalBugLog(BaseModel):
    bug_id: str = Field(description="Unique identifier for the bug")
    description: str = Field(description="Description of the bug")
    severity: int = Field(description="Severity of the bug, from 1 to 10")
    steps_to_reproduce: str = Field(description="Steps to reproduce the bug")
    actual_behavior: str = Field(description="Actual behavior observed")
    expected_behavior: str = Field(description="Expected behavior")
    system_info: str = Field(description="System information where the bug was observed")
    timestamp: datetime = Field(default_factory=datetime.now, description="Timestamp of the bug report")
    status: Optional[str] = Field(default="open", description="Status of the bug (e.g., open, closed, in progress)")

structured_llm = llm.with_structured_output(TechnicalBugLog)



In [38]:
system_prompt = """You are an AI assistant specialized in analyzing web session logs to help developers reproduce and understand bugs. When given a session recording log (consisting of timestamped user actions, network requests, and errors), analyze it carefully and produce:

A clear, step-by-step reproduction sequence for the observed issue
Identification of the error patterns and their potential root causes
Technical context about the errors (e.g., API failures, JSON parsing issues)
Relevant environment information extracted from the logs

Focus on extracting:

User navigation paths (URLs visited)
User interactions (clicks, form inputs)
Error messages and their patterns
Network request failures
The sequence and timing of events

Format your response as a formal bug report with:

Environment details
Clear numbered reproduction steps
Observed issues section with technical details
Additional observations that might help developers understand the context
If possible, identify whether the issue is likely client-side or server-side

Present the information in a structured, technical manner suitable for engineering teams, but ensure it remains understandable to non-technical stakeholders who may need to prioritize the issues."""

In [35]:
human_prompt = """This data was extracted from a Chrome extension that provides information about actions performed by the user and the elements: {data}"""

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt),
    ("user", human_prompt)],
)